In [7]:
"""
Prepare the DNA dataset for training, 
"""
import os
import pickle
import requests
import numpy as np

In [13]:
from Bio import Entrez, SeqIO
import os

def download_sequences(email, search_term, max_size_mb, batch_size=1000):
    Entrez.email = email
    handle = Entrez.esearch(db="nucleotide", term=search_term, retmax=batch_size)
    record = Entrez.read(handle)
    handle.close()

    ids = record["IdList"]
    downloaded_size = 0
    dataset = ""
    
    for start in range(0, len(ids), batch_size):
        end = min(len(ids), start+batch_size)
        print(f"Downloading records {start+1} to {end}...")

        fetch_handle = Entrez.efetch(db="nucleotide", id=ids[start:end], rettype="fasta", retmode="text")
        data = fetch_handle.read()
        fetch_handle.close()

        filename = f"sequences_{start+1}_to_{end}.fasta"
        with open(filename, "w") as f:
            f.write(data)

        downloaded_size += len(data)

        dataset += data
        if downloaded_size >= max_size_mb * 1024 * 1024:
            break

        print(f"Downloaded so far: {downloaded_size / (1024 * 1024):.2f} MB")
    return dataset

data = download_sequences("aaron@riedling.eu", "gene[All Fields]", 100)

Downloaded so far: 11.57 MB


In [14]:
len(data)

12135785

In [22]:
# get all the unique characters that occur in this text
chars = sorted(list(set(data)))
vocab_size = len(chars)
print("all the unique characters:", ''.join(chars))
print(f"vocab size: {vocab_size:,}")
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }

all the unique characters: 
 ,.0123456789;>ABCDEGJLNOPRSTUWXZ_abcdeghilmnopqrstuvwy
vocab size: 56


In [23]:
def encode(s):
    return [stoi[c] for c in s] # encoder: take a string, output a list of integers
def decode(l):
    return ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# create the train and test splits
n = len(data)
train_data = data[:int(n*0.9)]
val_data = data[int(n*0.9):]

# encode both to integers
train_ids = encode(train_data)
val_ids = encode(val_data)
print(f"train has {len(train_ids):,} tokens")
print(f"val has {len(val_ids):,} tokens")

train has 10,922,206 tokens
val has 1,213,579 tokens


In [33]:
# export to bin files
wd = current_dir = os.getcwd()
print("working directory:", wd)
dir_path = os.path.dirname(wd)
if not os.path.exists(dir_path):
    os.makedirs(dir_path)
train_ids = np.array(train_ids, dtype=np.uint16)
val_ids = np.array(val_ids, dtype=np.uint16)
train_ids.tofile(os.path.join(os.path.dirname(wd), 'train.bin'))
val_ids.tofile(os.path.join(os.path.dirname(wd), 'val.bin'))

# save the meta information as well, to help us encode/decode later
meta = {
    'vocab_size': vocab_size,
    'itos': itos,
    'stoi': stoi,
}
with open(os.path.join(os.path.dirname(wd), 'meta.pkl'), 'wb') as f:
    pickle.dump(meta, f)

# length of dataset in characters:  1115394
# all the unique characters:
#  !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
# vocab size: 65
# train has 1003854 tokens
# val has 111540 tokens

working directory: /home/aaron/DataspellProjects/nanoGPT/data/dna_gpt
